In [2]:
import ee
import geemap
import os

ee.Authenticate(force=True, scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/cloud-platform'])

# Initialize the library
ee.Initialize()
import numpy as np
from PIL import Image


Successfully saved authorization token.


In [3]:
# ==========================================
# 1. LOAD DHS CLUSTERS
# ==========================================

# Define paths and names
ASSET_ID = 'projects/integrated-hawk-485001-k3/assets/ph_dhs_sample_50'

output_dir = '/Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample'

# Load sample clusters
clusters = ee.FeatureCollection(ASSET_ID)

In [4]:
# ==========================================
# 2. DEFINE VIIRS PARAMETERS
# ==========================================
# Select 'average' radiance and create a single image from the collection
viirs_img = ee.ImageCollection("NOAA/VIIRS/DNB/ANNUAL_V22") \
    .filterDate('2022-01-01', '2022-12-31') \
    .select('average') \
    .median()

In [ ]:
# ==========================================
# 3. PROCESS AND EXPORT
# ==========================================
# Convert the collection to a list to iterate in Python
cluster_list = clusters.getInfo()['features']

for feature in cluster_list:
    props = feature['properties']
    cluster_id = props.get('DHSCLUST')
    
    # Logic for buffer size based on your project requirements
    # 'U' for Urban (2km), 'R' for Rural (5km)
    is_urban = props.get('URBAN_RURA') == 'U'
    radius = 2000 if is_urban else 5000
    
    # Create the bounding box for the chip
    lon, lat = feature['geometry']['coordinates']
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(radius).bounds()
    
    filename = os.path.join(output_dir, f'VIIRS_Cluster_{cluster_id}.tif')
    
    print(f"Downloading {filename}...")
    
    # This function downloads the image directly to local machine
    geemap.ee_export_image(
        viirs_img,
        filename=filename,
        scale=463,
        region=roi,
        file_per_band=False
    )

print("All chips downloaded to your local folder.")

Submitting 50 export tasks to Google Drive...
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_441.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_450.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_442.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_491.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_512.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_558.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/ruben/Desktop/Thesis/TrainingData/VIIRS/sample/VIIRS_Cluster_563.tif
Generating URL ...
Please wait ...
Data down